In [484]:
import warnings
import itertools
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
import pandas as pd
import statsmodels.api as sm
import matplotlib
matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12
#matplotlib.rcParams['text.color'] = 'G'
df = pd.read_excel("прогноз.xlsx")

Нулевая гипотеза расширенного теста Дики-Фуллера (ADF) заключается в том, что временной ряд не является стационарным. Таким образом, если p-значение теста ADF меньше уровня значимости (0,05), то мы отвергнем нулевую гипотезу и сделаем вывод, что временной ряд определенно стационарен. Как мы видим, p-значение более значимо, чем уровень значимости. если данные стационарны - преобразование бокса кокса


In [485]:
# проверка является ли ряд стационарным

from statsmodels.tsa.stattools import adfuller 
from numpy import log 
import pandas as pd 
 
mydata = pd.read_excel('прогноз.xlsx', names = ['value'], header = 0) 
 
res = adfuller( mydata.value.dropna()) 
print('Augmented Dickey-Fuller Statistic: %f' % res[0]) 
print('p-value: %f' % res[1])


Augmented Dickey-Fuller Statistic: -1.997011
p-value: 0.287885


In [ ]:
import numpy as np
y = np.log(y)
y

In [ ]:
# проверка является ли ряд стационарным

from statsmodels.tsa.stattools import adfuller 
from numpy import log 
import pandas as pd 
 
mydata = pd.read_excel('прогноз.xlsx', names = ['value'], header = 0) 
mydata = np.log(mydata)
res = adfuller( mydata.value.dropna()) 
print('Augmented Dickey-Fuller Statistic: %f' % res[0]) 
print('p-value: %f' % res[1])


In [ ]:
y = df.set_index(["trunc(create_date,'rm')"])
y.head(5)

In [ ]:
y.plot(figsize=(19, 4))
plt.show() 

In [ ]:
#трендб шум сезон
from pylab import rcParams
rcParams['figure.figsize'] = 18, 8
decomposition = sm.tsa.seasonal_decompose(y, model='additive')
fig = decomposition.plot()
plt.show()

In [ ]:
#подбираем параметры для модели
p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]
print('Examples of parameter for SARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

In [ ]:
#запускаем перебеор
for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(y,order=param,seasonal_order=param_seasonal,enforce_stationarity=False,enforce_invertibility=False)
            results = mod.fit()
            print('ARIMA{}x{}12 - AIC:{}'.format(param,param_seasonal,results.aic))
        except: 
            continue

In [ ]:
mod = sm.tsa.statespace.SARIMAX(y,
                                order=(0, 1, 1),
                                seasonal_order=(1, 1, 0,12 ),
                                #enforce_stationarity=False,
                                enforce_invertibility=False)
results = mod.fit()
print(results.summary().tables[1])


In [ ]:
#диагностика
results.plot_diagnostics(figsize=(18, 8))
plt.show()

In [ ]:
# тест на исторических данных 
pred = results.get_prediction(start=pd.to_datetime('2021-01-01'), dynamic=False)
pred_ci = pred.conf_int()
ax = y['2019':].plot(label='observed')
pred.predicted_mean.plot(ax=ax, label='One-step ahead Forecast', alpha=.7, figsize=(14, 4))
ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.2)
ax.set_xlabel('Date')
ax.set_ylabel('NPS')
plt.legend()
plt.show()

In [ ]:
pd.DataFrame(pred.predicted_mean) 

In [ ]:
# ошибка
y_forecasted = pred.predicted_mean
y_truth = y['2021-01-01':]
mse = ((y_forecasted - y_truth.nps) ** 2).mean()
print('The Mean Squared Error is {}'.format(round(mse, 5)))
print('The Root Mean Squared Error is {}'.format(round(np.sqrt(mse), 3)))

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_truth.nps , y_forecasted)

In [ ]:
pred_uc = results.get_forecast(steps=12)
pred_ci = pred_uc.conf_int()
ax = y.plot(label='observed', figsize=(14, 4))
pred_uc.predicted_mean.plot(ax=ax, label='Forecast')
ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.20)
ax.set_xlabel('Date')
ax.set_ylabel('nps')
plt.legend()
plt.show()

In [ ]:
pred_uc.predicted_mean

In [ ]:
y_forecasted = pred.predicted_mean
y_forecasted.head(12)

In [ ]:
y_truth.head(12)

In [ ]:
a = pred_ci.head(12)
a

In [ ]:
#прогноз
forecast = pred_uc.predicted_mean.head(12)
forecast = pd.DataFrame(forecast)
forecast

In [ ]:
d =a.join(forecast)
d

In [ ]:
y

In [363]:
# разобьём данные на обучающую и тестовую выборки

# обучающая выборка будет включать данные до декабря 1959 года включительно
train = y[:'2022-01-01']

# тестовая выборка начнется с января 1960 года (по сути, один год)
test = y['2022-01-01':]

In [ ]:
# выведем эти данные на графике
plt.plot(train, color = "black")
plt.plot(test, color = "red")

# заголовок и подписи к осям
plt.title('Разделение данных о перевозках на обучающую и тестовую выборки')
plt.ylabel('Количество пассажиров')
plt.xlabel('Месяцы')

# добавим сетку
plt.grid()

plt.show()

In [382]:
# принудительно отключим предупреждения системы
import warnings
warnings.simplefilter(action = 'ignore', category = Warning)

# обучим модель с соответствующими параметрами, SARIMAX(3, 0, 0)x(0, 1, 0, 12)

# импортируем класс модели
from statsmodels.tsa.statespace.sarimax import SARIMAX

# создадим объект этой модели
model = SARIMAX(train, 
                order = (3, 0, 0), 
                seasonal_order = (0, 1, 0, 12))

# применим метод fit
result = model.fit()

In [ ]:
# мы можем посмотреть результат с помощью метода summary()
print(result.summary())

In [ ]:
# тестовый прогнозный период начнется с конца обучающего периода
start = len(train)

# и закончится в конце тестового
end = len(train) + len(test) - 1
  
# применим метод predict
predictions = result.predict(start, end)
predictions

In [ ]:
# выведем три кривые (обучающая, тестовая выборка и тестовый прогноз)
plt.plot(train, color = 'black')
plt.plot(test, color = 'red')
plt.plot(predictions, color = 'green')

# заголовок и подписи к осям
plt.title('Обучающая выборка, тестовая выборка и тестовый прогноз')
plt.ylabel('Количество пассажиров')
plt.xlabel('Месяцы')

# добавим сетку
plt.grid()

plt.show()

In [ ]:
# импортируем метрику
from sklearn.metrics import mean_squared_error

# рассчитаем MSE
print(mean_squared_error(test, predictions))

# и RMSE
print(np.sqrt(mean_squared_error(test, predictions)))

In [ ]:
# прогнозный период начнется с конца имеющихся данных
start = len(y)

# и закончится 36 месяцев спустя
end = (len(y) - 1) + 1 * 12

# теперь построим прогноз на три года вперед
forecast = result.predict(start, end)

# посмотрим на весь 1963 год
forecast[-12:]

In [ ]:
# выведем две кривые (фактические данные и прогноз на будущее)
plt.plot(y, color = 'black')
plt.plot(forecast, color = 'blue')

# заголовок и подписи к осям
plt.title('Фактические данные и прогноз на будущее')
plt.ylabel('Количество пассажиров')
plt.xlabel('Месяцы')

# добавим сетку
plt.grid()

plt.show()

In [ ]:
# импортируем функцию для автоматического подбора параметров модели ARIMA
from pmdarima import auto_arima

  
# "погасим" предупреждения
import warnings
warnings.filterwarnings("ignore")
  
# настроим поиск параметров на обучающей выборке
parameter_search = auto_arima(train, start_p = 1, start_q = 1, max_p = 3, max_q = 3, m = 12, start_P = 0, seasonal = True,
                         d = None, D = 1, trace = True, error_action ='ignore', suppress_warnings = True,  stepwise = True)           #
  
# выведем результат
print(parameter_search.summary())

In [ ]:
https://kanoki.org/2020/04/30/time-series-analysis-and-forecasting-with-arima-python/

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(y[:200].nps)

In [ ]:
decomposition = sm.tsa.seasonal_decompose(y.nps, model='additive')
plt.rcParams["figure.figsize"] = [16,9]
fig = decomposition.plot()

In [393]:
from statsmodels.tsa.stattools import adfuller
def check_stationarity(timeseries):
    result = adfuller(timeseries,autolag='AIC')
    dfoutput = pd.Series(result[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    print('The test statistic: %f' % result[0])
    print('p-value: %f' % result[1])
    print('Critical Values:')
    for key, value in result[4].items():
        print('%s: %.3f' % (key, value))

In [ ]:
check_stationarity(y.nps)

In [ ]:
import numpy as np
ts_temp_log = np.log(y)
ts_temp_log

In [ ]:
ts_temp_log_diff = np.diff(ts_temp_log.nps)

plt.plot(ts_temp_log_diff)

In [ ]:
check_stationarity(ts_temp_log_diff)

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
plot_acf(ts_temp_log_diff,lags=10)

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf
plt.figure(figsize=(10,2))
plot_pacf(ts_temp_log_diff,lags=10)

In [ ]:
from pmdarima import auto_arima
auto_arima_fit = auto_arima(y, start_p=1, start_q=1,
                             max_p=3, max_q=3, m=12,
                             start_P=0, seasonal=True,
                             d=1, D=1, trace=True,
                             error_action='ignore',
                             suppress_warnings=True,
                             stepwise=True)

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

mod = SARIMAX(y.nps, order=(1,0,1))
results = mod.fit()
print(results.summary())

In [ ]:
residuals = pd.DataFrame(results.resid)
residuals.plot()

In [ ]:
residuals.plot(kind='kde')


In [ ]:
residuals.describe()

In [ ]:
plot_pacf(residuals)

In [ ]:
from math import sqrt
from sklearn.metrics import mean_squared_error
plt.plot(y.nps[:40],color='r')
plt.plot(results.predict(0,50),color='g')

In [ ]:
rmse =sqrt(mean_squared_error(y.nps, results.predict()))
print(rmse)

In [ ]:
n=36
forecast,err,ci = results.forecast(steps=n, 0.05)
df_forecast = pd.DataFrame({'forecast':forecast},index=pd.date_range(start='1/1/2020', periods=n, freq='MS'))